In [82]:
import pandas as pd
import numpy as np
from scipy import sparse
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
pd.options.display.max_rows = 250

# Importing Data

In [21]:
# Information about departments 
names = ['dept', 'deptdesc', 'trash']
use = names[0:2]
dept_df = pd.read_csv('/Users/aarij/Desktop/python-projects/iems-308/dillards/DillardsPOS/deptinfo.csv', sep= ",",
                     header=None, names=names, usecols=use)

In [22]:
# Information about stores
names = ['store', 'city', 'state', 'zip', 'trash']
use = names[0:4]
store_df = pd.read_csv('/Users/aarij/Desktop/python-projects/iems-308/dillards/DillardsPOS/strinfo.csv', sep= ",",
                     header=None, names=names, usecols=use)

In [23]:
# Information about individual SKUs 
names = ['sku', 'dept', 'classid', 'upc', 'style', 'color', 'size', 'packsize', 'vendor', 'brand', 'trash']
use = ['sku', 'dept', 'vendor', 'brand']
sku_df = pd.read_csv('/Users/aarij/Desktop/python-projects/iems-308/dillards/DillardsPOS/skuinfo.csv', sep= ",",
                     header=None, names=names, usecols=use, dtype={'vendor': str, 'brand': str})

In [24]:
# Information about what stores have what SKUs 
names = ['sku', 'store', 'cost', 'retail', 'trash']
use = ['sku', 'store', 'cost', 'retail']
skst_df = pd.read_csv('/Users/aarij/Desktop/python-projects/iems-308/dillards/DillardsPOS/skstinfo.csv', sep=',', 
                      header=None, names=names, usecols=use)

In [25]:
# Information about transactions
names = ['sku', 'store', 'register', 'tran', 'seq', 'date', 'stype', 'trash', 'quantity', 'unsure2', 'unsure3', 
         'interid', 'mic', 'trash2']
use = ['sku', 'store', 'register', 'tran', 'date']

transact_df = pd.read_csv('/Users/aarij/Desktop/python-projects/iems-308/dillards/DillardsPOS/trnsact.csv', sep= ",",
                     header=None, names=names, usecols=use)

# Exploratory Data Analysis

### Departments

In [26]:
dept_df

,dept,deptdesc
0,800,CLINIQUE
1,801,LESLIE
2,1100,GARY F
3,1107,JACQUES
4,1202,CABERN
5,1301,BE2
6,1704,R LAUREN
7,1905,R & Y
8,2102,CAB
9,2105,R TAYLOR


In [27]:
num_dept = dept_df.shape[0]
print(f'There are {num_dept} different departments')

There are 60 different departments


### Stores

In [28]:
number_stores = store_df.shape[0]
num_states = len(set(store_df.state))
top_five = store_df.state.value_counts()[0:5]

In [29]:
print(f'There are {number_stores} stores')
print(f'They are in {num_states} different states')
print(f'The five states where there are the most Dillards are \n \nState Count \n{top_five}')

There are 453 stores
They are in 31 different states
The five states where there are the most Dillards are 
 
State Count 
TX    79
FL    48
AR    27
AZ    26
OH    25
Name: state, dtype: int64


### SKUs

In [30]:
number_skus = sku_df.shape[0]
number_vendors = len(set(sku_df.vendor))
number_brands = len(set(sku_df.brand))

In [31]:
print(f'There are {number_skus} different SKUs')
print(f'They come from {number_vendors} different vendors')
print(f'Where in total there are {number_brands} different brands')

There are 1564178 different SKUs
They come from 2393 different vendors
Where in total there are 1960 different brands


### Store SKUs 

In [32]:
# Create a column of profit margins 
skst_df['profit'] = skst_df['retail'] - skst_df['cost']

In [33]:
# Sorting by highest profit SKUs 
skst_df.groupby('sku').mean().sort_values(by='profit', ascending=False)

,store,cost,retail,profit
sku,,,,
1811281,3354.833333,1872.54,5850.00,3977.46
1220149,1607.000000,1462.56,5280.00,3817.44
6200173,1607.000000,2700.00,6017.00,3317.00
5480674,1607.000000,1536.72,4776.00,3239.28
5240674,1607.000000,1536.72,4776.00,3239.28
...,...,...,...,...
9738777,5604.000000,660.00,191.76,-468.24
8759889,8409.000000,912.00,395.70,-516.30
2809844,8555.153846,900.00,269.82,-630.18


In [34]:
# Looking at different profit values
skst_df.groupby('sku').mean().describe()['profit']

count    760212.000000
mean         20.019637
std          77.545588
min        -710.320000
25%          -3.756502
50%           4.294452
75%          22.785820
max        3977.460000
Name: profit, dtype: float64

In [35]:
# Getting average profit for SKUs across all stores they're in
avg_profit = skst_df.groupby('sku').mean()['profit'].mean().round(2)

In [36]:
print(f'The average profit margin is ${avg_profit}')

The average profit margin is $20.02


### Transactions

In [37]:
# Transactions 
# Casting datetimes as dates
transact_df.date = pd.to_datetime(transact_df.date)

In [38]:
# Getting the range of dates we're analyzing 
# Note, this command takes very long to run
earliest_date = min(transact_df['date']).date()
latest_date = max(transact_df['date']).date()

In [39]:
# Creating a new index which we can group by to get baskets 
transact_df['index'] = transact_df['store'] + transact_df['register'] + transact_df['tran'] + transact_df['date']

In [41]:
num_basket = len(transact_df.groupby('index').index)

In [42]:
num_tran = transact_df.shape[0]

In [43]:
print(f"We're looking at dates between {earliest_date} and {latest_date}")
print(f"We have {num_tran} total transactions")
print(f"We have {num_basket} total baskets")

We're looking at dates between 2004-08-01 and 2005-08-27
We have 120916896 total transactions
We have 5028079 total baskets


# Subsetting and Preparing Data of Interest

In [44]:
# Looking only at transactions made in december
dec_df = transact_df[transact_df['date'].dt.month == 12]

In [45]:
# Getting the 200 most popular SKUs from those transactions
top_sku = dec_df['sku'].value_counts()[0:200].index.to_list()

In [46]:
# Looking at only those 1000 SKUs for those months
tdf = dec_df[dec_df['sku'].isin(top_sku)]

In [47]:
# Getting dummy variables for all different SKUs 
onehot = pd.get_dummies(tdf['sku'], prefix='sku')

In [48]:
# Concating the dummy variable to each line item in the transaction df
df = pd.concat([tdf, onehot], axis=1)

In [49]:
# Removing the unneccesary columns, but leave index
df.drop(['sku', 'store', 'register', 'tran', 'date'], axis=1, inplace=True)

In [50]:
# Important - here I sum over all indices so I can create the baskets. A basket will have a 0 for all items it 
# doesn't have and a 1 for all items it does 
finaldf = df.groupby(['index']).sum()

In [51]:
# Convert to booleans for faster loading times 
finaldf = finaldf.astype(dtype=bool)

# Creating Association Rules

In [52]:
# After testing, a minsup of 0.004 allows us to see many rules without crashing the kernel 
frequentItemsets = apriori(finaldf, min_support=0.004, use_colnames=True)

In [53]:
frequentItemsets

,support,itemsets
0,0.036046,(sku_173088)
1,0.011351,(sku_176136)
2,0.013843,(sku_208362)
3,0.028288,(sku_247668)
4,0.023679,(sku_264715)
...,...,...
314,0.004833,"(sku_8146822, sku_8166822)"
315,0.005146,"(sku_8156822, sku_8166822)"
316,0.004584,"(sku_8616048, sku_8956048)"
317,0.006252,"(sku_3898011, sku_3978011, sku_3524026)"


In [54]:
rules = association_rules(frequentItemsets, metric="lift", min_threshold=1)

In [77]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(sku_3949538),(sku_173088),0.065407,0.036046,0.004120,0.062989,1.747471,0.001762,1.028754
1,(sku_173088),(sku_3949538),0.036046,0.065407,0.004120,0.114297,1.747471,0.001762,1.055199
2,(sku_803921),(sku_3524026),0.038214,0.075473,0.005910,0.154651,2.049092,0.003026,1.093663
3,(sku_3524026),(sku_803921),0.075473,0.038214,0.005910,0.078303,2.049092,0.003026,1.043495
4,(sku_803921),(sku_3898011),0.038214,0.036852,0.004033,0.105551,2.864159,0.002625,1.076805
...,...,...,...,...,...,...,...,...,...
241,"(sku_3968011, sku_3690654)",(sku_3898011),0.006997,0.036852,0.004901,0.700463,19.007312,0.004644,3.215457
242,"(sku_3898011, sku_3690654)",(sku_3968011),0.006414,0.028947,0.004901,0.764177,26.398822,0.004716,4.117725
243,(sku_3968011),"(sku_3898011, sku_3690654)",0.028947,0.006414,0.004901,0.169321,26.398822,0.004716,1.196113
244,(sku_3898011),"(sku_3968011, sku_3690654)",0.036852,0.006997,0.004901,0.133001,19.007312,0.004644,1.145333


In [83]:
rules.sort_values(by='lift', ascending=False).reset_index(drop=True)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(sku_8616048),(sku_8956048),0.004584,0.404255,33.428231
1,(sku_8956048),(sku_8616048),0.004584,0.379095,33.428231
2,(sku_3968011),"(sku_3898011, sku_3690654)",0.004901,0.169321,26.398822
3,"(sku_3898011, sku_3690654)",(sku_3968011),0.004901,0.764177,26.398822
4,(sku_8156822),(sku_8166822),0.005146,0.342604,21.249234
5,(sku_8166822),(sku_8156822),0.005146,0.319187,21.249234
6,(sku_8156822),(sku_8146822),0.004743,0.315752,20.761697
7,(sku_8146822),(sku_8156822),0.004743,0.311864,20.761697
8,(sku_8146822),(sku_8166822),0.004833,0.317784,19.709821
9,(sku_8166822),(sku_8146822),0.004833,0.299754,19.709821
